## Домашнее задание «Рекомендации на основе скрытых факторов»
### Никифоров Владимир

-  “Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

In [3]:
!pip install implicit

    100% |████████████████████████████████| 1.1MB 9.0MB/s 
    100% |████████████████████████████████| 51kB 6.7MB/s 
  Stored in directory: /home/truename/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit
  Found existing installation: tqdm 4.26.0
    Uninstalling tqdm-4.26.0:
      Successfully uninstalled tqdm-4.26.0
You are using pip version 19.0.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

In [51]:
raw_data = pd.read_table('../data/hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat', usecols=['userID','bookmarkID','tagID'])

In [52]:
print(raw_data.describe())
raw_data.head()

              userID     bookmarkID          tagID
count  437593.000000  437593.000000  437593.000000
mean    46120.551432   40635.829952    5869.114463
std     32450.744004   31627.970529   10467.301443
min         8.000000       1.000000       1.000000
25%     14601.000000   13355.000000     339.000000
50%     43978.000000   30575.000000    1132.000000
75%     74110.000000   66245.000000    5739.000000
max    108035.000000  107253.000000   52955.000000


,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,1
3,8,7,6
4,8,7,7


In [53]:
# Вместо tagID будем считать количество tag-ов
raw_data = raw_data.groupby(['userID', 'bookmarkID'])['tagID'].count().reset_index()
raw_data.columns = ['userID', 'bookmarkID', 'tagCount']
raw_data.head()

,userID,bookmarkID,tagCount
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2


In [54]:
rows = raw_data.userID.astype(int)
cols = raw_data.bookmarkID.astype(int)
tags = list(raw_data.tagCount)

In [55]:
data_sparse = sparse.csr_matrix((tags, (cols, rows)), shape=(max(cols)+1, max(rows)+1))

In [56]:
model = AlternatingLeastSquares(factors=100)
model.fit(data_sparse)

In [58]:
list(raw_data['userID'].unique())

[8,
 32,
 57,
 147,
 233,
 242,
 259,
 302,
 319,
 336,
 372,
 427,
 485,
 521,
 538,
 620,
 788,
 819,
 913,
 934,
 970,
 1016,
 1033,
 1049,
 1094,
 1103,
 1113,
 1149,
 1263,
 1296,
 1320,
 1328,
 1352,
 1423,
 1455,
 1476,
 1505,
 1557,
 1701,
 1751,
 1774,
 1822,
 1856,
 1887,
 1933,
 1952,
 1987,
 2022,
 2032,
 2055,
 2128,
 2151,
 2232,
 2300,
 2315,
 2330,
 2367,
 2393,
 2437,
 2462,
 2487,
 2498,
 2549,
 2570,
 2586,
 2611,
 2636,
 2701,
 2716,
 2748,
 2831,
 2879,
 2911,
 2934,
 2952,
 2961,
 2989,
 3032,
 3083,
 3122,
 3203,
 3220,
 3233,
 3257,
 3279,
 3337,
 3396,
 3421,
 3454,
 3513,
 3539,
 3568,
 3646,
 3728,
 3743,
 3789,
 3817,
 3836,
 3914,
 3943,
 3957,
 4031,
 4089,
 4109,
 4135,
 4154,
 4188,
 4237,
 4324,
 4363,
 4488,
 4549,
 4681,
 4717,
 4742,
 4762,
 4795,
 4828,
 4911,
 4977,
 5024,
 5046,
 5075,
 5108,
 5186,
 5251,
 5266,
 5323,
 5424,
 5478,
 5584,
 5606,
 5648,
 5802,
 5817,
 5863,
 5937,
 5947,
 5959,
 6020,
 6047,
 6067,
 6092,
 6111,
 6120,
 6141,
 61

In [70]:
userid = 259

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)
print(recommendations)

[(662, 0.6809554), (661, 0.67981803), (611, 0.6352436), (649, 0.63493264), (610, 0.63038224), (626, 0.6123637), (635, 0.5933938), (629, 0.5820317), (664, 0.5779096), (608, 0.5774816)]


In [66]:
titles = pd.read_table('../data/hetrec2011-delicious-2k/bookmarks.dat', encoding='koi8-r', usecols=['id','title'])
titles.head()

,id,title
0,1,IFLA - The official website of the Internation...
1,2,gcdp-e.pdf (application/pdf Object)
2,7,EdSelect
3,8,Cool Canada (Collections Canada)
4,9,Kidsreads.com


In [74]:
# Итого - рекоммендация внести в закладки следующие страницы:
titles[titles['id'].isin([r[0] for r in recommendations])]

,id,title
443,608,Information Architecture 101: Techniques and B...
445,610,Common Security Mistakes in Web Applications -...
446,611,Entitifier
458,626,Showcase of Interesting Navigation Designs - S...
461,629,An Analysis of Typography on the Web | Nettuts+
466,635,"Free Wireframing Kits, UI Design Kits, PDFs an..."
479,649,Commonly Confused Bits Of jQuery - Smashing Ma...
489,661,A Complete Guide To Tumblr - Smashing Magazine
490,662,11 Best WebSites To Make Your Job Interview Su...
492,664,A List Apart: Articles: SVG with a little help...
